In [6]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import tiny_face_model
import util
from argparse import ArgumentParser
import cv2
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle

import pylab as pl
import time
import os
import sys
from scipy.special import expit
import glob

MAX_INPUT_DIM = 5000.0

def overlay_bounding_boxes(raw_img, refined_bboxes, lw):
 

  # Overlay bounding boxes on an image with the color based on the confidence.
  count=0
  for r in refined_bboxes:
    _score = expit(r[4])
    cm_idx = int(np.ceil(_score * 255))
    rect_color = [int(np.ceil(x * 255)) for x in util.cm_data[cm_idx]]  # parula
    _lw = lw
    if lw == 0:  # line width of each bounding box is adaptively determined.
      bw, bh = r[2] - r[0] + 1, r[3] - r[0] + 1
      _lw = 1 if min(bw, bh) <= 20 else max(2, min(3, min(bh / 20, bw / 20)))
      _lw = int(np.ceil(_lw * _score))

    _r = [int(x) for x in r[:4]]
    cv2.rectangle(raw_img, (_r[0], _r[1]), (_r[2], _r[3]), rect_color, _lw)
    count+=1
  print("--------------NUMBER OF TINY FACES IN THE IMAGE--------------------: ",count)
    
def evaluate(weight_file_path, data_dir, output_dir, prob_thresh=0.5, nms_thresh=0.1, lw=3, display=False):


  # placeholder of input images. Currently batch size of one is supported.
  x = tf.placeholder(tf.float32, [1, None, None, 3]) # n, h, w, c

  # Create the tiny face model which weights are loaded from a pretrained model.
  model = tiny_face_model.Model(weight_file_path)
  score_final = model.tiny_face(x)

  # Find image files in data_dir.
  filenames = []
  for ext in ('*.png', '*.gif', '*.jpg', '*.jpeg'):
    filenames.extend(glob.glob(os.path.join(data_dir, ext)))

  # Load an average image and clusters(reference boxes of templates).
  with open(weight_file_path, "rb") as f:
    _, mat_params_dict = pickle.load(f)

  average_image = model.get_data_by_key("average_image")
  clusters = model.get_data_by_key("clusters")
  clusters_h = clusters[:, 3] - clusters[:, 1] + 1
  clusters_w = clusters[:, 2] - clusters[:, 0] + 1
  normal_idx = np.where(clusters[:, 4] == 1)

  # main
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for filename in filenames:
      fname = filename.split(os.sep)[-1]
      raw_img = cv2.imread(filename)
      raw_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)
      raw_img_f = raw_img.astype(np.float32)

      def _calc_scales():
        raw_h, raw_w = raw_img.shape[0], raw_img.shape[1]
        min_scale = min(np.floor(np.log2(np.max(clusters_w[normal_idx] / raw_w))),
                        np.floor(np.log2(np.max(clusters_h[normal_idx] / raw_h))))
        max_scale = min(1.0, -np.log2(max(raw_h, raw_w) / MAX_INPUT_DIM))
        scales_down = np.arange(min_scale, 0, 1.)
        scales_up = np.arange(0.5, max_scale, 0.5)
        scales_pow = np.hstack((scales_down, scales_up))
        scales = np.power(2.0, scales_pow)
        return scales

      scales = _calc_scales()
      start = time.time()

      # initialize output
      bboxes = np.empty(shape=(0, 5))

      # process input at different scales
      for s in scales:
        print("Processing {} at scale {:.4f}".format(fname, s))
        img = cv2.resize(raw_img_f, (0, 0), fx=s, fy=s, interpolation=cv2.INTER_LINEAR)
        img = img - average_image
        img = img[np.newaxis, :]

        # we don't run every template on every scale ids of templates to ignore
        tids = list(range(4, 12)) + ([] if s <= 1.0 else list(range(18, 25)))
        ignoredTids = list(set(range(0, clusters.shape[0])) - set(tids))

        # run through the net
        score_final_tf = sess.run(score_final, feed_dict={x: img})

        # collect scores
        score_cls_tf, score_reg_tf = score_final_tf[:, :, :, :25], score_final_tf[:, :, :, 25:125]
        prob_cls_tf = expit(score_cls_tf)
        prob_cls_tf[0, :, :, ignoredTids] = 0.0

        def _calc_bounding_boxes():
          # threshold for detection
          _, fy, fx, fc = np.where(prob_cls_tf > prob_thresh)

          # interpret heatmap into bounding boxes
          cy = fy * 8 - 1
          cx = fx * 8 - 1
          ch = clusters[fc, 3] - clusters[fc, 1] + 1
          cw = clusters[fc, 2] - clusters[fc, 0] + 1

          # extract bounding box refinement
          Nt = clusters.shape[0]
          tx = score_reg_tf[0, :, :, 0:Nt]
          ty = score_reg_tf[0, :, :, Nt:2*Nt]
          tw = score_reg_tf[0, :, :, 2*Nt:3*Nt]
          th = score_reg_tf[0, :, :, 3*Nt:4*Nt]

          # refine bounding boxes
          dcx = cw * tx[fy, fx, fc]
          dcy = ch * ty[fy, fx, fc]
          rcx = cx + dcx
          rcy = cy + dcy
          rcw = cw * np.exp(tw[fy, fx, fc])
          rch = ch * np.exp(th[fy, fx, fc])

          scores = score_cls_tf[0, fy, fx, fc]
          tmp_bboxes = np.vstack((rcx - rcw / 2, rcy - rch / 2, rcx + rcw / 2, rcy + rch / 2))
          tmp_bboxes = np.vstack((tmp_bboxes / s, scores))
          tmp_bboxes = tmp_bboxes.transpose()
          return tmp_bboxes

        tmp_bboxes = _calc_bounding_boxes()
        bboxes = np.vstack((bboxes, tmp_bboxes)) # <class 'tuple'>: (5265, 5)


      print("time {:.2f} secs for {}".format(time.time() - start, fname))

      # non maximum suppression
      # refind_idx = util.nms(bboxes, nms_thresh)
      refind_idx = tf.image.non_max_suppression(tf.convert_to_tensor(bboxes[:, :4], dtype=tf.float32),
                                                   tf.convert_to_tensor(bboxes[:, 4], dtype=tf.float32),
                                                   max_output_size=bboxes.shape[0], iou_threshold=nms_thresh)
      refind_idx = sess.run(refind_idx)
      refined_bboxes = bboxes[refind_idx]
      overlay_bounding_boxes(raw_img, refined_bboxes, lw)

      if display:
        # plt.axis('off')
        plt.imshow(raw_img)
        plt.show()

      # save image with bounding boxes
      raw_img = cv2.cvtColor(raw_img, cv2.COLOR_RGB2BGR)
      cv2.imwrite(os.path.join(output_dir, fname), raw_img)

def main():

  argparse = ArgumentParser()
  argparse.add_argument('--weight_file_path', type=str, help='Pretrained weight file.', default="./path/to/hr_res101.pkl")
  argparse.add_argument('--data_dir', type=str, help='Image data directory.', default="./path/to/input_image_directory")
  argparse.add_argument('--output_dir', type=str, help='Output directory for images with faces detected.', default="./path/to/output_directory")
  argparse.add_argument('--prob_thresh', type=float, help='The threshold of detection confidence(default: 0.5).', default=0.5)
  argparse.add_argument('--nms_thresh', type=float, help='The overlap threshold of non maximum suppression(default: 0.1).', default=0.1)
  argparse.add_argument('--line_width', type=int, help='Line width of bounding boxes(0: auto).', default=3)
  argparse.add_argument('--display', type=bool, help='Display each image on window.', default=True)

  args = argparse.parse_args()

  # check arguments
  assert os.path.exists(args.weight_file_path), "weight file: " + args.weight_file_path + " not found."
  assert os.path.exists(args.data_dir), "data directory: " + args.data_dir + " not found."
  assert os.path.exists(args.output_dir), "output directory: " + args.output_dir + " not found."
  assert args.line_width >= 0, "line_width should be >= 0."

  with tf.Graph().as_default():
    evaluate(
      weight_file_path=args.weight_file_path, data_dir=args.data_dir, output_dir=args.output_dir,
      prob_thresh=args.prob_thresh, nms_thresh=args.nms_thresh,
      lw=args.line_width, display=args.display)

if __name__ == '__main__':
  main()


ModuleNotFoundError: ignored

In [4]:
argparse = ArgumentParser()
argparse.add_argument('--matlab_model_path', type=str, help='Matlab pretrained model.',
                      default='/path/to/hr_res101.mat')
argparse.add_argument('--weight_file_path', type=str, help='Weight file for Tensorflow.',
                      default='/path/to/mat2tf.pkl')

args = argparse.parse_args()

# check arguments
assert os.path.exists(args.matlab_model_path), \
    "Matlab pretrained model: " + args.matlab_model_path + " not found."
assert os.path.exists(os.path.dirname((args.weight_file_path))),\
    "Directory for weight file for Tensorflow: " + args.weight_file_path + " not found."

mat_params_dict = {}
mat_blocks_dict = {}

f = sio.loadmat(args.matlab_model_path)
net = f['net']
clusters = np.copy(net['meta'][0][0][0][0][6])
average_image = np.copy(net['meta'][0][0][0][0][2][0][0][2])[:, 0]
mat_params_dict["clusters"] = clusters
mat_params_dict["average_image"] = average_image

layers = net['layers'][0][0][0]
mat_params = net['params'][0][0][0]
for p in mat_params:
    mat_params_dict[p[0][0]] = p[1]

for k, layer in enumerate(layers):
    type_string = ''
    param_string = ''

    layer_name, layer_type = layer[0][0], layer[1][0]
    layer_inputs = []
    layer_outputs = []
    layer_params = []

    layer_inputs_count = layer[2][0].shape[0]
    for i in range(layer_inputs_count):
        layer_inputs.append(layer[2][0][i][0])

    layer_outputs_count = layer[3][0].shape[0]
    for i in range(layer_outputs_count):
        layer_outputs.append(layer[3][0][i][0])

    if layer[4].shape[0] > 0:
        layer_params_count = layer[4][0].shape[0]
        for i in range(layer_params_count):
            layer_params.append(layer[4][0][i][0])

    mat_blocks_dict[layer_name + '_type'] = layer_type
    mat_params_dict[layer_name + '_type'] = layer_type
    if layer_type == u'dagnn.Conv':
        nchw = layer[5][0][0][0][0]
        has_bias = layer[5][0][0][1][0][0]
        pad = layer[5][0][0][3][0]
        stride = layer[5][0][0][4][0]
        dilate = layer[5][0][0][5][0]
        mat_blocks_dict[layer_name + '_nchw'] = nchw
        mat_blocks_dict[layer_name + '_has_bias'] = has_bias
        mat_blocks_dict[layer_name + '_pad'] = pad
        mat_blocks_dict[layer_name + '_stride'] = stride
        mat_blocks_dict[layer_name + '_dilate'] = dilate
        if has_bias:
            bias = mat_params_dict[layer_name + '_bias'][0] # (1, N) -> (N,)
            mat_params_dict[layer_name + '_bias'] = bias
    elif layer_type == u'dagnn.BatchNorm':
        epsilon = layer[5][0][0][1][0][0]
        gamma = mat_params_dict[layer_name + '_mult'][:, 0] # (N, 1) -> (N,)
        beta = mat_params_dict[layer_name + '_bias'][:, 0] # (N, 1) -> (N,)
        moments = mat_params_dict[layer_name + '_moments'] # (N, 2) -> (N,), (N,)
        moving_mean = moments[:, 0]
        moving_var = moments[:, 1] * moments[:, 1] - epsilon

        mat_blocks_dict[layer_name + '_variance_epsilon'] = epsilon
        mat_params_dict[layer_name + '_scale'] = gamma
        mat_params_dict[layer_name + '_offset'] = beta
        mat_params_dict[layer_name + '_mean'] = moving_mean
        mat_params_dict[layer_name + '_variance'] = moving_var
    elif layer_type == u'dagnn.ConvTranspose':
        nchw = layer[5][0][0][0][0]
        has_bias = layer[5][0][0][1][0][0]
        upsample = layer[5][0][0][2][0]
        crop = layer[5][0][0][3][0]
        mat_blocks_dict[layer_name + '_nchw'] = nchw
        mat_blocks_dict[layer_name + '_has_bias'] = has_bias
        mat_blocks_dict[layer_name + '_upsample'] = upsample
        mat_blocks_dict[layer_name + '_crop'] = crop
        wmat = mat_params_dict[layer_name + 'f']
        mat_params_dict[layer_name + '_filter'] = wmat
    elif layer_type == u'dagnn.Pooling':
        method = layer[5][0][0][0][0]
        pool_size = layer[5][0][0][1][0]
        pad = layer[5][0][0][3][0]
        stride = layer[5][0][0][4][0]
        mat_blocks_dict[layer_name + '_method'] = method
        mat_blocks_dict[layer_name + '_pool_size'] = pool_size
        mat_blocks_dict[layer_name + '_pad'] = pad
        mat_blocks_dict[layer_name + '_stride'] = stride
    elif layer_type == u'dagnn.ReLU':
        pass
    elif layer_type == u'dagnn.Sum':
        pass
    else:
        pass

with open(args.weight_file_path, 'wb') as f:
    pickle.dump([mat_blocks_dict, mat_params_dict], f, pickle.HIGHEST_PROTOCOL)


usage: ipykernel_launcher.py [-h] [--matlab_model_path MATLAB_MODEL_PATH]
                             [--weight_file_path WEIGHT_FILE_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-60ff6b28-21aa-4049-aa72-7a8bbd38a3ed.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
